<a href="https://colab.research.google.com/github/mahdimalverdi/ACM/blob/master/Exam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
spark = SparkSession.builder.master("local[1]").appName("hw6").getOrCreate()

In [5]:
!wget https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/dataset2.csv

--2021-06-22 13:02:35--  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/dataset2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18696924 (18M) [text/plain]
Saving to: ‘dataset2.csv’

dataset2.csv        100%[===================>]  17.83M  57.5MB/s    in 0.3s    

2021-06-22 13:02:36 (57.5 MB/s) - ‘dataset2.csv’ saved [18696924/18696924]



In [6]:
train_dataset = spark.read.csv("dataset2.csv",inferSchema=True, header =True)
train_dataset.printSchema()

root
 |-- action: string (nullable = true)
 |-- created_at: double (nullable = true)
 |-- source_event_id: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- post_page_offset: integer (nullable = true)
 |-- tokens: string (nullable = true)
 |-- post_index_in_post_list: integer (nullable = true)
 |-- post_token: string (nullable = true)



In [7]:
import re

def strip_margin(text):
    nomargin = re.sub('\n[ \t]*\|', ' ', text)
    trimmed = re.sub('\s+', ' ', nomargin)
    return trimmed

In [8]:
train_dataset.createOrReplaceTempView("df")
d = spark.sql(strip_margin("select (select count(*) from (SELECT count(*) FROM df group by source_event_id having count(*) > 9))/ count(*) from (SELECT count(*), source_event_id FROM df group by source_event_id)"))
d.show()

+-------------------------------------------------------------+
|(CAST(scalarsubquery() AS DOUBLE) / CAST(count(1) AS DOUBLE))|
+-------------------------------------------------------------+
|                                           0.2271264689423615|
+-------------------------------------------------------------+



In [9]:
train_dataset.createOrReplaceTempView("df")
d = spark.sql(strip_margin("select (select count(*) from (SELECT distinct source_event_id from df where source_event_id not in (SELECT source_event_id FROM df where action = 'click_post' group by source_event_id))) / count(*) from (SELECT count(*), source_event_id FROM df group by source_event_id)"))
d.show()

+-------------------------------------------------------------+
|(CAST(scalarsubquery() AS DOUBLE) / CAST(count(1) AS DOUBLE))|
+-------------------------------------------------------------+
|                                          0.24314493564633463|
+-------------------------------------------------------------+



In [10]:
train_dataset.createOrReplaceTempView("df")
d = spark.sql(strip_margin("select df1.source_event_id, df1.count / df2.count * 100 from (SELECT count(*) as count , source_event_id FROM df   where action = 'click_post' group by source_event_id)as df1 inner join (SELECT count(*) as count , source_event_id FROM df group by source_event_id) as df2 on df2.source_event_id = df1.source_event_id"))
d.show()

+--------------------+-----------------------------------------------------------------------+
|     source_event_id|((CAST(count AS DOUBLE) / CAST(count AS DOUBLE)) * CAST(100 AS DOUBLE))|
+--------------------+-----------------------------------------------------------------------+
|98719974-7821-455...|                                                                  100.0|
|dab24ff1-c7ed-4dd...|                                                                  100.0|
|9460ceb5-d423-498...|                                                                  100.0|
|388e35f9-8262-47a...|                                                                  100.0|
|87f449de-dd59-4ea...|                                                      70.58823529411765|
|690ca150-5b6d-4aa...|                                                      74.28571428571429|
|0eed1d54-2715-448...|                                                                  100.0|
|5ee25cf4-a97a-457...|                            

In [11]:
train_dataset.createOrReplaceTempView("df")
d = spark.sql(strip_margin("(SELECT min(post_index_in_post_list)  , source_event_id FROM df   where action = 'click_post' group by source_event_id)"))
d.show()

+----------------------------+--------------------+
|min(post_index_in_post_list)|     source_event_id|
+----------------------------+--------------------+
|                           8|98719974-7821-455...|
|                           3|dab24ff1-c7ed-4dd...|
|                           1|9460ceb5-d423-498...|
|                          41|388e35f9-8262-47a...|
|                           5|87f449de-dd59-4ea...|
|                          14|690ca150-5b6d-4aa...|
|                           1|0eed1d54-2715-448...|
|                          37|5ee25cf4-a97a-457...|
|                           1|9f63595b-331c-494...|
|                        1749|9ba63c1c-9cb9-48d...|
|                           8|ebc8b59b-61fa-40a...|
|                          26|249198b4-4cbb-4c3...|
|                         147|67c61e88-1562-4c6...|
|                          30|70d4b496-f9ee-4ac...|
|                          45|f88ae8bd-edb9-471...|
|                           1|5c5e6964-3472-4d0...|
|           

In [12]:
train_dataset.createOrReplaceTempView("df")
d = spark.sql(strip_margin(""))
d.show()


ParseException: ignored

In [ ]:
d.show()

+-------------------------------------------------------------+
|(CAST(scalarsubquery() AS DOUBLE) / CAST(count(1) AS DOUBLE))|
+-------------------------------------------------------------+
|                                           0.2271264689423615|
+-------------------------------------------------------------+

